# 协程
- 参考资料
   - http://python.jobbole.com/86481
   - http://python.jobbole.com/87310
   - https://sementfault.com/a/1190000009781688
   
   ----

# 迭代器
   - 可迭代（Iterable）:直接作用于for循环的变量
   - 迭代器（Iterator）:不但可以作用于for循环，还可以被next调用

In [2]:
# 可迭代
l = [i for i in range(10)]
# l 是迭代对象
for idx in l:
    print(idx)
    
# range() 是一个迭代器
for i in range(4):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [3]:
# isinstance 判断某个变量是否是一个实例

# 判断是否可迭代
from collections import Iterable

l = [1,2,3]
# 是否是可迭代对象
isinstance(l,Iterable)

True

In [4]:
from collections import Iterator
# 是否是可迭代对象
isinstance(l,Iterator)

False

In [7]:
# 通过Iter函数 转换
s = 'I am Marlon'

print(isinstance(s,Iterable))
print(isinstance(s,Iterator))

# 转换成可迭代对象
s_iter = iter(s)
print(isinstance(s_iter,Iterable))
print(isinstance(s_iter,Iterator))

True
False
True
True


---

# 生产器
   - generator 一边循环，一边计算下一个元素的机制/算法
   - 需要满足三个条件
      - 每次调用都生产出一个for循环需要的下一个元素或者
      - 如果达到了最后一个后，报出StopIteration异常
      - 可以被next函数调用
   - 如何生产一个generator
      - 直接使用
      - 如果函数中包含 yield，则这个函数就叫做生成器
      - next 调用函数，遇到 yield返回

In [14]:
# 直接使用生成器

# 这是列表对象
L = [x*x for x in range(5)]
print(type(L))

# 这是一个列表生成器对象
l =(x*x for x in range(5))
print(type(l))

<class 'list'>
<class 'generator'>


In [35]:
# 包含 yield函数的生成器

# normal 
def odd():
    print("Step 1")
    print("Step 2")
    print("Step 3")
    return None
odd()



Step 1
Step 2
Step 3


In [38]:
# yield 
def odd1():
    print("Step 1")
    yield 1
    print("Step 2")
    yield 2
    print("Step 3")
    yield 3

g = odd1()

# add() 是调用生成器 
one = next(g)
print(one)

two = next(g)
print(two)

three = next(g)
print(three)

Step 1
1
Step 2
2
Step 3
3


In [42]:
# for 循环调用生成器

def fib(max):
    n,a,b = 0,0,1
    while n < max:
        print(b)
        a,b = b, a+b
        n += 1
    return 'Done'

fib(5)

1
1
2
3
5


'Done'

In [45]:
# 斐波那契数列 的生成器写法

def fib2(max):
    n,a,b = 0,0,1
    while n < max:
        yield b
        a,b = b, a+b
        n += 1
    return 'Done'

ge = fib2(5)
for i in range(6):
    r = next(ge)
    print(r)

1
1
2
3
5


StopIteration: Done

In [46]:
# yield写法2, 在循环中使用
'''
生成器的典型用法就是在for中使用
比较常用的典型生成器，就是range
'''
g2 = fib2(10)
for i in g2:
    print(i)

1
1
2
3
5
8
13
21
34
55


# 协程
- 历史历程
   - 3.4 引入协程，用yield实现
   - 3.5 引入协程语法
   - 实现卸程比较好的包有 asyncio,tornado,gevent
- 定义：协程 是为了非抢占式多任务产生子程序的计算机程序组件，协程允许不用的入口点在不同的位置暂停或者开始执行程序
- 从技术角度讲，协程就是一个你可以暂停执行的函数，或者干脆把协程理解成生成器
- 协程的实现：
   - yield 返回
   - send 调用
   
- 协程的四个状态：
   - inspect.getgeneratorstate(...)函数确定
      1. GEN_CREATED:
      2. GEN_RUNNING:
      3. GEN_SUSPEND:
      4. GEN_CLOSED:
      - next 预激（prime）
      - 代码案例
- 协程的终止：
   - 协程中未处理的异常会向上冒泡，传给next函数，或者send方法的调用方（即触发协程的对象）
   - 终止协程的一种方式：发送某个哨符值，让协程退出，内置的None客Elliise等常量经常用做哨符值 
   
- yield from
   - 调用协程是为了得到返回值，协程必须正常终止
   - 生成器正常终止会发出StopIteration的异常，异常对象的value属性保存返回值
   - yield from 从内部捕获StopIteration异常
   - 案例 v03
   - 委派生成器：
      - 包含yield from表达式的生成器函数
      - 委派生成器在yield from表达式处暂停，调用方可以直接把数据发送给子生成器
      - 子生成器再把产出的值发送调用方
      - 子生成器在最后，解释器会抛出 StopIteration异常,并把返回值给

In [49]:
# 协程代码的案例1
def simple_coroutine():
    print('-> start')
    x = yield
    print ('-> receved ',x)

#   主线程
sc = simple_coroutine()
print(1111)

next(sc) #预激
print(2222)

# 使用send也可以，x会受到该参数
sc.send('marlon')

1111
-> start
2222
-> receved  marlon


StopIteration: 

In [51]:
# 协程状态的案例

def simplle_coroutine2(a):
    print('- > start')
    b = yield a
    print('- > received ',a,b)
    
    c = yield a+b
    print("- > received ",a,b,c)
    yield a+b+c

sc = simplle_coroutine2(5)

aa = next(sc)
print(aa)

bb = sc.send(6)
print(bb)

cc = sc.send(7)
print(cc)

- > start
5
- > received  5 6
11
- > received  5 6 7
18


In [52]:
# 案例 v03
def gen():
    for c in 'AB':
        yield c

print(list(gen()))

def gen_new():
    yield from 'AB'
print(list(gen_new()))

['A', 'B']
['A', 'B']


In [56]:
# 委派生成器的案例
from collections import namedtuple


'''
解释：
1 外出
'''
ResClass = namedtuple('Res','count average')

# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None
    
    while True:
        term = yield
        
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return ResClass(count,average)

# 委派生成器
def grouper(storages,key):
    while True:
#         获取averager()返回值
        storages[key] = yield from averager()

#  客户端代码
def client():
    process_data = {
        'boys_1':[39.0,48.8,43.2,40.8,43.1,38.6,41.4,40.6,36.3],
        'boys_2':[1.38,1.5,1.32,1.25,1.37,1.48,1.25,1.49,1.46]
    }
    
    storages = {}
    for k,v in process_data.items():
        crountine = grouper(storages,k)
        
        next(crountine)
        
        for dt in v:
            crountine.send(dt)
        
        crountine.send(None)
    print(storages)

#  run
client()
        

{'boys_1': Res(count=9, average=41.31111111111111), 'boys_2': Res(count=9, average=1.3888888888888888)}
